In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [2]:
vid = pd.read_csv('./videos-stats-raw.csv',  index_col=0)
com = pd.read_csv('./comments-raw.csv', index_col=0)

In [3]:
vid.head()

,Title,Video ID,Published At,Keyword,Likes,Comments,Views
0,Apple Pay Is Killing the Physical Wallet After...,wAZZ-UWGVHI,2022-08-23,tech,3407.0,672.0,135612.0
1,The most EXPENSIVE thing I own.,b3x28s61q3c,2022-08-24,tech,76779.0,4306.0,1758063.0
2,My New House Gaming Setup is SICK!,4mgePWWCAmA,2022-08-23,tech,63825.0,3338.0,1564007.0
3,Petrol Vs Liquid Nitrogen | Freezing Experimen...,kXiYSI7H2b0,2022-08-23,tech,71566.0,1426.0,922918.0
4,Best Back to School Tech 2022!,ErMwWXQxHp0,2022-08-08,tech,96513.0,5155.0,1855644.0


In [4]:
com.head()

,Video ID,Comment,Likes,Sentiment
0,wAZZ-UWGVHI,Let's not forget that Apple Pay in 2014 requir...,95.0,1.0
1,wAZZ-UWGVHI,Here in NZ 50% of retailers don’t even have co...,19.0,0.0
2,wAZZ-UWGVHI,I will forever acknowledge this channel with t...,161.0,2.0
3,wAZZ-UWGVHI,Whenever I go to a place that doesn’t take App...,8.0,0.0
4,wAZZ-UWGVHI,"Apple Pay is so convenient, secure, and easy t...",34.0,2.0


In [5]:
vid.rename(columns={'Video ID': 'id', 'Published At': 'date'}, inplace=True)
vid.columns = vid.columns.str.lower()
vid = vid[['id', 'title', 'date', 'keyword', 'views', 'likes', 'comments']]
com.rename(columns={'Video ID': 'id'}, inplace=True)
com.columns = com.columns.str.lower()

In [6]:
vid.isna().sum()

id          0
title       0
date        0
keyword     0
views       2
likes       2
comments    2
dtype: int64

In [7]:
com.isna().sum()

id           0
comment      1
likes        0
sentiment    0
dtype: int64

In [8]:
vid.dropna(inplace=True)
com.dropna(inplace=True)

In [9]:
vid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1879 entries, 0 to 1878
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        1879 non-null   object 
 1   title     1879 non-null   object 
 2   date      1879 non-null   object 
 3   keyword   1879 non-null   object 
 4   views     1879 non-null   float64
 5   likes     1879 non-null   float64
 6   comments  1879 non-null   float64
dtypes: float64(3), object(4)
memory usage: 117.4+ KB


In [10]:
com.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18408 entries, 0 to 18408
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         18408 non-null  object 
 1   comment    18408 non-null  object 
 2   likes      18408 non-null  float64
 3   sentiment  18408 non-null  float64
dtypes: float64(2), object(2)
memory usage: 719.1+ KB


In [11]:
# merge vid with com (row: 1 com, 1 vid)
com_vid = pd.merge(com, vid, on='id')
com_vid.rename(columns= {'likes_x':'com_likes', 'title':'vid_title', 'likes_y':'vid_likes', 'comments':'vid_comments',
                        'comments':'vid_comments', 'views' :'vid_views', 'sentiment':'com_sentiment'}, inplace=True)
com_vid.head()

,id,comment,com_likes,com_sentiment,vid_title,date,keyword,vid_views,vid_likes,vid_comments
0,wAZZ-UWGVHI,Let's not forget that Apple Pay in 2014 requir...,95.0,1.0,Apple Pay Is Killing the Physical Wallet After...,2022-08-23,tech,135612.0,3407.0,672.0
1,wAZZ-UWGVHI,Here in NZ 50% of retailers don’t even have co...,19.0,0.0,Apple Pay Is Killing the Physical Wallet After...,2022-08-23,tech,135612.0,3407.0,672.0
2,wAZZ-UWGVHI,I will forever acknowledge this channel with t...,161.0,2.0,Apple Pay Is Killing the Physical Wallet After...,2022-08-23,tech,135612.0,3407.0,672.0
3,wAZZ-UWGVHI,Whenever I go to a place that doesn’t take App...,8.0,0.0,Apple Pay Is Killing the Physical Wallet After...,2022-08-23,tech,135612.0,3407.0,672.0
4,wAZZ-UWGVHI,"Apple Pay is so convenient, secure, and easy t...",34.0,2.0,Apple Pay Is Killing the Physical Wallet After...,2022-08-23,tech,135612.0,3407.0,672.0


In [12]:
# join all comments, get mean metrics and 
# merge com with vid  (row: 1 vid, many comments)
for i in com.id.unique():
    vid.loc[vid.id==i, 'comment'] = ' || '.join(com.loc[com.id==i, 'comment'])
    vid.loc[vid.id==i, 'mean_comment_likes'] = com.loc[com.id==i, 'likes'].mean()
    vid.loc[vid.id==i, 'mean_comment_sentiment'] = com.loc[com.id==i, 'sentiment'].mean()
vid.head()

,id,title,date,keyword,views,likes,comments,comment,mean_comment_likes,mean_comment_sentiment
0,wAZZ-UWGVHI,Apple Pay Is Killing the Physical Wallet After...,2022-08-23,tech,135612.0,3407.0,672.0,Let's not forget that Apple Pay in 2014 requir...,39.1,1.2
1,b3x28s61q3c,The most EXPENSIVE thing I own.,2022-08-24,tech,1758063.0,76779.0,4306.0,"Wow, you really went to town on the PSU test r...",598.2,1.8
2,4mgePWWCAmA,My New House Gaming Setup is SICK!,2022-08-23,tech,1564007.0,63825.0,3338.0,Linus!!! Just turn the key lights 180 and bou...,626.2,1.9
3,kXiYSI7H2b0,Petrol Vs Liquid Nitrogen | Freezing Experimen...,2022-08-23,tech,922918.0,71566.0,1426.0,Unstoppable experiments with liquid nitrogen 🎉...,528.8,1.6
4,ErMwWXQxHp0,Best Back to School Tech 2022!,2022-08-08,tech,1855644.0,96513.0,5155.0,"Guys, a quick note that you do NOT need all th...",2721.7,1.6


In [20]:
vid['unicode_title'] = vid.title.astype('U')
vid['unicode_comment'] = vid.comment.astype('U')

vid['likes_per_view'] = vid.likes / vid.views
vid['comments_per_view'] = vid.comments / vid.views

vid['views_above_mean'] = vid.views > vid.views.mean()
vid['likes_above_mean'] = vid.likes > vid.likes.mean()
vid['comments_above_mean'] = vid.comments > vid.comments.mean()

vid.date = pd.to_datetime(vid.date)
vid['days_old'] = (vid.date.max() - vid.date).dt.days

In [21]:
vid.head()

,id,title,date,keyword,views,likes,comments,comment,mean_comment_likes,mean_comment_sentiment,likes_per_view,comments_per_view,views_above_mean,likes_above_mean,comments_above_mean,days_old,unicode_title,unicode_comment
0,wAZZ-UWGVHI,Apple Pay Is Killing the Physical Wallet After...,2022-08-23,tech,135612.0,3407.0,672.0,Let's not forget that Apple Pay in 2014 requir...,39.1,1.2,0.025123,0.004955,False,False,False,1,Apple Pay Is Killing the Physical Wallet After...,Let's not forget that Apple Pay in 2014 requir...
1,b3x28s61q3c,The most EXPENSIVE thing I own.,2022-08-24,tech,1758063.0,76779.0,4306.0,"Wow, you really went to town on the PSU test r...",598.2,1.8,0.043672,0.002449,False,False,False,0,The most EXPENSIVE thing I own.,"Wow, you really went to town on the PSU test r..."
2,4mgePWWCAmA,My New House Gaming Setup is SICK!,2022-08-23,tech,1564007.0,63825.0,3338.0,Linus!!! Just turn the key lights 180 and bou...,626.2,1.9,0.040809,0.002134,False,False,False,1,My New House Gaming Setup is SICK!,Linus!!! Just turn the key lights 180 and bou...
3,kXiYSI7H2b0,Petrol Vs Liquid Nitrogen | Freezing Experimen...,2022-08-23,tech,922918.0,71566.0,1426.0,Unstoppable experiments with liquid nitrogen 🎉...,528.8,1.6,0.077543,0.001545,False,False,False,1,Petrol Vs Liquid Nitrogen | Freezing Experimen...,Unstoppable experiments with liquid nitrogen 🎉...
4,ErMwWXQxHp0,Best Back to School Tech 2022!,2022-08-08,tech,1855644.0,96513.0,5155.0,"Guys, a quick note that you do NOT need all th...",2721.7,1.6,0.052011,0.002778,False,False,False,16,Best Back to School Tech 2022!,"Guys, a quick note that you do NOT need all th..."


In [15]:
num_cols = ['days_old', 'views', 'likes', 'comments', 'mean_comment_likes', 'mean_comment_sentiment', 'likes_per_view', 'comments_per_view']

norm_vid = vid.copy()
norm = MinMaxScaler()
norm.fit(norm_vid[num_cols])
norm_vid[num_cols] = norm.transform(norm_vid[num_cols])

In [17]:
norm_vid.head()

,id,title,date,keyword,views,likes,comments,comment,mean_comment_likes,mean_comment_sentiment,likes_per_view,comments_per_view,views_above_mean,likes_above_mean,comments_above_mean,days_old
0,wAZZ-UWGVHI,Apple Pay Is Killing the Physical Wallet After...,2022-08-23,tech,0.000034,0.000207,0.000918,Let's not forget that Apple Pay in 2014 requir...,0.000225,0.60,0.115269,0.059800,False,False,False,0.000181
1,b3x28s61q3c,The most EXPENSIVE thing I own.,2022-08-24,tech,0.000436,0.004669,0.005877,"Wow, you really went to town on the PSU test r...",0.003439,0.90,0.200102,0.029772,False,False,False,0.000000
2,4mgePWWCAmA,My New House Gaming Setup is SICK!,2022-08-23,tech,0.000388,0.003881,0.004556,Linus!!! Just turn the key lights 180 and bou...,0.003600,0.95,0.187005,0.025998,False,False,False,0.000181
3,kXiYSI7H2b0,Petrol Vs Liquid Nitrogen | Freezing Experimen...,2022-08-23,tech,0.000229,0.004352,0.001947,Unstoppable experiments with liquid nitrogen 🎉...,0.003040,0.80,0.355006,0.018938,False,False,False,0.000181
4,ErMwWXQxHp0,Best Back to School Tech 2022!,2022-08-08,tech,0.000460,0.005869,0.007036,"Guys, a quick note that you do NOT need all th...",0.015645,0.80,0.238235,0.033711,False,False,False,0.002900


In [23]:
vid.to_csv('./videos-with-comments-aggr.csv')
norm_vid.to_csv('./norm-videos-with-comments-aggr.csv')
com_vid.to_csv('./comments-with-video-stats.csv')